# Working with virtualized resources on Grid'5000

When working with bare-metal machines isn't enough.

EnOSlib uses `Providers` to ... provide resources. They transform an abstract resource configuration into a concrete one.
To do so, they interact with an infrastructure where they get the resources from. There are different providers in EnOSlib: 

- Vbox/KVM to work with locally hosted virtual machines
- Openstack/Chameleon to work with bare-metal resources hosted in the Chameleon platform
- FiT/IOT lab to work with sensors or low profile machines
- **VmonG5k to work with virtual machines on Grid'5000**
- **Distem to work with lxc containers on Grid'5000**
- G5k, of course

The purpose of the above is to ease the use of the platform by internalizing some of the configuration tasks (e.g automatically managing the reservation on G5k, network configuration ...)

In the following we'll cover some of the EnOSlib way of managing virtual machines on Grid'5000, docker containers or lxc containers on Grid'5000.

---

- Website: https://discovery.gitlabpages.inria.fr/enoslib/index.html
- Instant chat: https://framateam.org/enoslib
- Source code: https://gitlab.inria.fr/discovery/enoslib

---

**Prerequisites:**

- A Grid'5000 account
- A working EnOSlib environment and Jupyter (not included in EnOSlib dependencies, but `pip install jupyterlab` will install it)




## Virtual Machines

The VMonG5K provider which provides a quick way to start virtual machines for you on Grid'5000. 

This setup is opinionated and follows these steps

- First, the number of required physical machine is computed (based on cpu/ram demands) and then they are reserved
- Second, a subnet is reserved (/22 or /16), which will be used as a pool of available MAC/IP 
- Third, the virtual machines are distributed on the physical machines and assigned a MAC/IP.
- Fourth, the virtual machines are started

---
The following configuration wil start 10 VMs with different roles.

In [13]:
import enoslib as en

# get some logging info
import logging
logging.basicConfig(level=logging.INFO)

# claim the resources
conf = (
    en.VMonG5kConf
    .from_settings(job_name="enoslib_providers")
    .add_machine(
        roles=["compute"],
        cluster="paravance",
        number=8,
        flavour_desc={
            "core": 2,
            "mem": 2048
        }
    )
    .add_machine(
        roles=["controler"],
        cluster="paravance",
        number=2,
        flavour="tiny"
    )
    .finalize()
)


provider = en.VMonG5k(conf)

roles, networks = provider.init()
print(roles)
print(networks)

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from grenoble
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Submitting {'name': 'enoslib_providers', 'types': ['allow_classic_ssh'], 'resources': "{cluster='paravance'}/nodes=1+{cluster='paravance'}/nodes=1+slash_22=1,walltime=02:00:00", 'command': 'sleep 31536000', 'queue': 'default'} on rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1820218 o


PLAY [This is a play for virtual machines on Grid'5000] *************************************************************************************************************************************

TASK [Gathering Facts] **********************************************************************************************************************************************************************
ok: [paravance-15.rennes.grid5000.fr]
ok: [paravance-14.rennes.grid5000.fr]

TASK [Enable nested virtualization] *********************************************************************************************************************************************************
changed: [paravance-14.rennes.grid5000.fr]
changed: [paravance-15.rennes.grid5000.fr]

TASK [list only running VMs] ****************************************************************************************************************************************************************
ok: [paravance-15.rennes.grid5000.fr]
ok: [paravance-14.rennes.grid5000.fr

In [14]:
roles

{'compute': [VirtualMachine(address='10.158.0.2', alias='virtual-158-0-2', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.0.3', alias='virtual-158-0-3', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.0.4', alias='virtual-158-0-4', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.0.5', alias='virtual-158-0-5', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.0.6', alias='virtual-158-0-6', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.0.7', alias='virtual-158-0-7', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.0.8', alias='virtual-158-0-8', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.0.9', alias='virtual-158-0-9', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices='')], 'controler': [VirtualMachine(address='10.158.0.10', alias='virtual-158-0-10', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices=''), VirtualMachine(address='10.158.0.11', alias='virtual-158-0-11', user='root', keyfile=None, port=None, extra={'gateway': 'access.grid5000.fr', 'gateway_user': 'msimonin'}, net_devices='')]}

In [15]:
networks

{'__subnet__': [<enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7fa7d0857d50>]}

In [16]:
# VMs can take some time to be reachable, let's wait for them
en.wait_for(roles)
roles = en.sync_info(roles, networks)


PLAY [all] **********************************************************************************************************************************************************************************

TASK [hostname] *****************************************************************************************************************************************************************************
 [started TASK: hostname on virtual-158-0-2]
 [started TASK: hostname on virtual-158-0-3]
 [started TASK: hostname on virtual-158-0-4]
 [started TASK: hostname on virtual-158-0-5]
 [started TASK: hostname on virtual-158-0-6]
 [started TASK: hostname on virtual-158-0-7]
 [started TASK: hostname on virtual-158-0-8]
 [started TASK: hostname on virtual-158-0-9]
 [started TASK: hostname on virtual-158-0-10]
 [started TASK: hostname on virtual-158-0-11]
changed: [virtual-158-0-3]
changed: [virtual-158-0-4]
changed: [virtual-158-0-2]
changed: [virtual-158-0-5]
changed: [virtual-158-0-11]
changed: [virtual-158-0-8]
cha

In [17]:
roles

ip
fe80::216:3eff:fe9e:2/64
10.158.0.2/14
ip
::1/128
127.0.0.1/8
ip
fe80::216:3eff:fe9e:3/64
10.158.0.3/14
ip
::1/128


In [18]:
networks

{'__subnet__': [<enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7fa7d0857d50>]}

In [19]:
results = en.run_command("nproc", roles=roles)
[(r.host,  r.stdout) for r in results]


PLAY [all] **********************************************************************************************************************************************************************************

TASK [nproc] ********************************************************************************************************************************************************************************
 [started TASK: nproc on virtual-158-0-2]
 [started TASK: nproc on virtual-158-0-3]
 [started TASK: nproc on virtual-158-0-4]
 [started TASK: nproc on virtual-158-0-5]
 [started TASK: nproc on virtual-158-0-6]
 [started TASK: nproc on virtual-158-0-7]
 [started TASK: nproc on virtual-158-0-8]
 [started TASK: nproc on virtual-158-0-9]
 [started TASK: nproc on virtual-158-0-10]
 [started TASK: nproc on virtual-158-0-11]
changed: [virtual-158-0-2]
changed: [virtual-158-0-3]
changed: [virtual-158-0-4]
changed: [virtual-158-0-5]
changed: [virtual-158-0-7]
changed: [virtual-158-0-8]
changed: [virtual-158-0-9]
changed

[('virtual-158-0-2', '2'),
 ('virtual-158-0-3', '2'),
 ('virtual-158-0-4', '2'),
 ('virtual-158-0-5', '2'),
 ('virtual-158-0-7', '2'),
 ('virtual-158-0-8', '2'),
 ('virtual-158-0-9', '2'),
 ('virtual-158-0-10', '1'),
 ('virtual-158-0-6', '2'),
 ('virtual-158-0-11', '1')]

In [20]:
provider.destroy()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from grenoble
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading 1820218 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_providers from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Killing the job (rennes, 1820218)


## Docker containers

There's no specific provider for manipulating Docker Container as hosts but some utility functions to help you.

- most of the time you don't need to change the internal state of docker containers 
  (you just fire up some applications using docker container and that's it)
- so this technique cover use cases where you need to use docker container instead a bare metal machine
  and needs to interact with the containerized linux distribution (installing software, configuring stuffs ...)
  
---
The strategy is the following:
- First reserve some bare-metal machines
- Install and start some docker containers
- Get the representation of the docker containers as Hosts

In [21]:
import logging
from pathlib import Path

import enoslib as en

logging.basicConfig(level=logging.DEBUG)

prod_network = en.G5kNetworkConf(
    type="prod", roles=["my_network"], site="rennes"
)
conf = (
    en.G5kConf.from_settings(job_name="enoslib_docker", job_type="allow_classic_ssh")
    .add_network_conf(prod_network)
    .add_machine(
        roles=["control"], cluster="paravance", nodes=1, primary_network=prod_network
    )
    .finalize()
)

provider = en.G5k(conf)

# Get actual resources
roles, networks = provider.init()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from grenoble
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading 1820217 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1820217 on rennes [2021-09-27 11:03:00]
INFO:enoslib.infra.enos_g5k.g5k_api_utils:All jobs are Running !


We install docker using the EnOSlib's service.

In [22]:
# Workaround (temporary)
with en.actions(roles=roles) as a:
    a.file(path="/etc/apt/sources.list.d/repo.radeon.com.list", state="absent")
 
# Install docker
d = en.Docker(agent=roles["control"], bind_var_docker="/tmp/docker")
d.deploy()


PLAY [all] **********************************************************************************************************************************************************************************

TASK [file] *********************************************************************************************************************************************************************************
 [started TASK: file on paravance-10.rennes.grid5000.fr]
changed: [paravance-10.rennes.grid5000.fr]


[WARNING]: Invalid characters were found in group names but not replaced, use -vvvv to see details



PLAY [Deploy docker and registry] ***********************************************************************************************************************************************************

TASK [Gathering Facts] **********************************************************************************************************************************************************************
ok: [paravance-10.rennes.grid5000.fr]

TASK [registry : include] *******************************************************************************************************************************************************************
included: /home/msimonin/workspace/repos/enoslib/enoslib/service/docker/roles/registry/tasks/deploy.yml for paravance-10.rennes.grid5000.fr

TASK [registry : include] *******************************************************************************************************************************************************************
included: /home/msimonin/workspace/repos/enoslib/enoslib/s

[WARNING]: Updating cache and auto-installing missing dependency: python3-apt


ok: [paravance-10.rennes.grid5000.fr]

TASK [registry : fix "No module named ssl_match_hostname" issue] ****************************************************************************************************************************
changed: [paravance-10.rennes.grid5000.fr]

TASK [registry : Installing docker python bindings] *****************************************************************************************************************************************
changed: [paravance-10.rennes.grid5000.fr]

TASK [registry : Creating docker state directory] *******************************************************************************************************************************************
changed: [paravance-10.rennes.grid5000.fr] => (item=/tmp/docker)
changed: [paravance-10.rennes.grid5000.fr] => (item=/var/lib/docker)

TASK [registry : Bind mount the docker volume directory] ***************************************************************************************************************

In [23]:
# Start some containers
N = 5
with en.play_on(roles=roles) as p:
    for i in range(N):
        p.docker_container(
            name=f"mydocker-{i}",
            image="ubuntu",
            state="started",
            command="sleep 10d",
        )


PLAY [all] **********************************************************************************************************************************************************************************

TASK [docker_container] *********************************************************************************************************************************************************************
 [started TASK: docker_container on paravance-10.rennes.grid5000.fr]
changed: [paravance-10.rennes.grid5000.fr]

TASK [docker_container] *********************************************************************************************************************************************************************
 [started TASK: docker_container on paravance-10.rennes.grid5000.fr]
changed: [paravance-10.rennes.grid5000.fr]

TASK [docker_container] *********************************************************************************************************************************************************************
 [started TA

In [24]:
dockers = en.get_dockers(roles=roles)


PLAY [*] ************************************************************************************************************************************************************************************

TASK [docker ps -q --filter name=.* | xargs docker inspect] *********************************************************************************************************************************
 [started TASK: docker ps -q --filter name=.* | xargs docker inspect on paravance-10.rennes.grid5000.fr]
changed: [paravance-10.rennes.grid5000.fr]


In [25]:
dockers

[DockerHost(address='/mydocker-4', alias='/mydocker-4-paravance-10.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={'ansible_connection': 'docker', 'ansible_docker_extra_args': '-H ssh://root@paravance-10.rennes.grid5000.fr', 'mitogen_via': 'root@paravance-10.rennes.grid5000.fr'}, net_devices=set()),
 DockerHost(address='/mydocker-3', alias='/mydocker-3-paravance-10.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={'ansible_connection': 'docker', 'ansible_docker_extra_args': '-H ssh://root@paravance-10.rennes.grid5000.fr', 'mitogen_via': 'root@paravance-10.rennes.grid5000.fr'}, net_devices=set()),
 DockerHost(address='/mydocker-2', alias='/mydocker-2-paravance-10.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={'ansible_connection': 'docker', 'ansible_docker_extra_args': '-H ssh://root@paravance-10.rennes.grid5000.fr', 'mitogen_via': 'root@paravance-10.rennes.grid5000.fr'}, net_devices=set()),
 DockerHost(address='/mydocker-1', alias='/

EnOSlib/Ansible requires python at the destination for many operations. However minimal docker images will lickely not include python...
Using the option `raw=True` can overcome this limitation: https://docs.ansible.com/ansible/latest/collections/ansible/builtin/raw_module.html

In [26]:
en.run_command("hostname", roles=dockers, raw=True)


PLAY [all] **********************************************************************************************************************************************************************************

TASK [hostname] *****************************************************************************************************************************************************************************
 [started TASK: hostname on /mydocker-4-paravance-10.rennes.grid5000.fr]
 [started TASK: hostname on /mydocker-3-paravance-10.rennes.grid5000.fr]
 [started TASK: hostname on /mydocker-2-paravance-10.rennes.grid5000.fr]
 [started TASK: hostname on /mydocker-1-paravance-10.rennes.grid5000.fr]
 [started TASK: hostname on /mydocker-0-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-1-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-4-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-3-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-2-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-0-paravance-1

CommandResult(host='/mydocker-1-paravance-10.rennes.grid5000.fr', task='hostname', status='OK', payload={'rc': 0, 'stdout': '3d900c5064c8\n', 'stdout_lines': ['3d900c5064c8'], 'stderr': '', 'stderr_lines': [], 'changed': True, '_ansible_no_log': False})
CommandResult(host='/mydocker-4-paravance-10.rennes.grid5000.fr', task='hostname', status='OK', payload={'rc': 0, 'stdout': 'a92212c40cc1\n', 'stdout_lines': ['a92212c40cc1'], 'stderr': '', 'stderr_lines': [], 'changed': True, '_ansible_no_log': False})
CommandResult(host='/mydocker-3-paravance-10.rennes.grid5000.fr', task='hostname', status='OK', payload={'rc': 0, 'stdout': '2b5f17a1ca07\n', 'stdout_lines': ['2b5f17a1ca07'], 'stderr': '', 'stderr_lines': [], 'changed': True, '_ansible_no_log': False})
CommandResult(host='/mydocker-2-paravance-10.rennes.grid5000.fr', task='hostname', status='OK', payload={'rc': 0, 'stdout': 'cff17e8f2e0c\n', 'stdout_lines': ['cff17e8f2e0c'], 'stderr': '', 'stderr_lines': [], 'changed': True, '_ansible_no_log': False})
CommandResult(host='/mydocker-0-paravance-10.rennes.grid5000.fr', task='hostname', status='OK', payload={'rc': 0, 'stdout': '6204d5aa4e56\n', 'stdout_lines': ['6204d5aa4e56'], 'stderr': '', 'stderr_lines': [], 'changed': True, '_ansible_no_log': False})

In [27]:
en.run_command("apt update && apt install -y python3", roles=dockers, raw=True)


PLAY [all] **********************************************************************************************************************************************************************************

TASK [apt update && apt install -y python3] *************************************************************************************************************************************************
 [started TASK: apt update && apt install -y python3 on /mydocker-4-paravance-10.rennes.grid5000.fr]
 [started TASK: apt update && apt install -y python3 on /mydocker-3-paravance-10.rennes.grid5000.fr]
 [started TASK: apt update && apt install -y python3 on /mydocker-2-paravance-10.rennes.grid5000.fr]
 [started TASK: apt update && apt install -y python3 on /mydocker-1-paravance-10.rennes.grid5000.fr]
 [started TASK: apt update && apt install -y python3 on /mydocker-0-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-2-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-0-paravance-10.rennes.grid5000.fr]
cha

CommandResult(host='/mydocker-2-paravance-10.rennes.grid5000.fr', task='apt update && apt install -y python3', status='OK', payload={'rc': 0, 'stdout': "Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]\nGet:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]\nGet:3 http://archive.ubuntu.com/ubuntu focal-backports InRelease [101 kB]\nGet:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]\nGet:5 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]\nGet:6 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]\nGet:7 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]\nGet:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [1537 kB]\nGet:9 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1071 kB]\nGet:10 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [33.3 kB]\nGet:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [590 kB]\nGet:12 http://archive.ubuntu.com/ubuntu focal-backports/main amd64 Packages [2668 B]\nGet:13 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]\nGet:14 http://archive.ubuntu.com/ubuntu focal-backports/universe amd64 Packages [6310 B]\nGet:15 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [30.1 kB]\nGet:16 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [543 kB]\nGet:17 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [791 kB]\nGet:18 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1092 kB]\nFetched 19.1 MB in 3s (7240 kB/s)\nReading package lists...\nBuilding dependency tree...\nReading state information...\n5 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  file libexpat1 libmagic-mgc libmagic1 libmpdec2 libpython3-stdlib\n  libpython3.8-minimal libpython3.8-stdlib libreadline8 libsqlite3-0 libssl1.1\n  mime-support python3-minimal python3.8 python3.8-minimal readline-common\n  xz-utils\nSuggested packages:\n  python3-doc python3-tk python3-venv python3.8-venv python3.8-doc binutils\n  binfmt-support readline-doc\nThe following NEW packages will be installed:\n  file libexpat1 libmagic-mgc libmagic1 libmpdec2 libpython3-stdlib\n  libpython3.8-minimal libpython3.8-stdlib libreadline8 libsqlite3-0 libssl1.1\n  mime-support python3 python3-minimal python3.8 python3.8-minimal\n  readline-common xz-utils\n0 upgraded, 18 newly installed, 0 to remove and 5 not upgraded.\nNeed to get 7393 kB of archives.\nAfter this operation, 32.8 MB of additional disk space will be used.\nGet:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libssl1.1 amd64 1.1.1f-1ubuntu2.8 [1320 kB]\nGet:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpython3.8-minimal amd64 3.8.10-0ubuntu1~20.04 [717 kB]\nGet:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libexpat1 amd64 2.2.9-1build1 [73.3 kB]\nGet:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3.8-minimal amd64 3.8.10-0ubuntu1~20.04 [1898 kB]\nGet:5 http://archive.ubuntu.com/ubuntu focal/main amd64 python3-minimal amd64 3.8.2-0ubuntu2 [23.6 kB]\nGet:6 http://archive.ubuntu.com/ubuntu focal/main amd64 mime-support all 3.64ubuntu1 [30.6 kB]\nGet:7 http://archive.ubuntu.com/ubuntu focal/main amd64 libmpdec2 amd64 2.4.2-3 [81.1 kB]\nGet:8 http://archive.ubuntu.com/ubuntu focal/main amd64 readline-common all 8.0-4 [53.5 kB]\nGet:9 http://archive.ubuntu.com/ubuntu focal/main amd64 libreadline8 amd64 8.0-4 [131 kB]\nGet:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsqlite3-0 amd64 3.31.1-4ubuntu0.2 [549 kB]\nGet:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpython3.8-stdlib amd64 3.8.10-0ubuntu1~20.04 [1675 kB]\nGet:12 http://archive.ubuntu.co

In [28]:
# we're now good to use raw=False (the default)
en.run_command("date", roles=dockers)


PLAY [all] **********************************************************************************************************************************************************************************

TASK [date] *********************************************************************************************************************************************************************************
 [started TASK: date on /mydocker-4-paravance-10.rennes.grid5000.fr]
 [started TASK: date on /mydocker-3-paravance-10.rennes.grid5000.fr]
 [started TASK: date on /mydocker-2-paravance-10.rennes.grid5000.fr]
 [started TASK: date on /mydocker-1-paravance-10.rennes.grid5000.fr]
 [started TASK: date on /mydocker-0-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-0-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-4-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-2-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-1-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-3-paravance-10.rennes.grid5000.fr

CommandResult(host='/mydocker-0-paravance-10.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Mon Sep 27 09:09:45 UTC 2021', 'stderr': '', 'rc': 0, 'start': '2021-09-27 09:09:45.873968', 'end': '2021-09-27 09:09:45.876690', 'delta': '0:00:00.002722', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Mon Sep 27 09:09:45 UTC 2021'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='/mydocker-4-paravance-10.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Mon Sep 27 09:09:45 UTC 2021', 'stderr': '', 'rc': 0, 'start': '2021-09-27 09:09:45.876622', 'end': '2021-09-27 09:09:45.879372', 'delta': '0:00:00.002750', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Mon Sep 27 09:09:45 UTC 2021'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='/mydocker-2-paravance-10.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Mon Sep 27 09:09:46 UTC 2021', 'stderr': '', 'rc': 0, 'start': '2021-09-27 09:09:46.307336', 'end': '2021-09-27 09:09:46.310213', 'delta': '0:00:00.002877', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Mon Sep 27 09:09:46 UTC 2021'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='/mydocker-1-paravance-10.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Mon Sep 27 09:09:46 UTC 2021', 'stderr': '', 'rc': 0, 'start': '2021-09-27 09:09:46.310129', 'end': '2021-09-27 09:09:46.313080', 'delta': '0:00:00.002951', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Mon Sep 27 09:09:46 UTC 2021'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='/mydocker-3-paravance-10.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Mon Sep 27 09:09:46 UTC 2021', 'stderr': '', 'rc': 0, 'start': '2021-09-27 09:09:46.415957', 'end': '2021-09-27 09:09:46.418667', 'delta': '0:00:00.002710', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Mon Sep 27 09:09:46 UTC 2021'], 'stderr_lines': [], '_ansible_no_log': False})

In [29]:
en.run_command("apt install -y iproute2", roles=dockers)



PLAY [all] **********************************************************************************************************************************************************************************

TASK [apt install -y iproute2] **************************************************************************************************************************************************************
 [started TASK: apt install -y iproute2 on /mydocker-4-paravance-10.rennes.grid5000.fr]
 [started TASK: apt install -y iproute2 on /mydocker-3-paravance-10.rennes.grid5000.fr]
 [started TASK: apt install -y iproute2 on /mydocker-2-paravance-10.rennes.grid5000.fr]
 [started TASK: apt install -y iproute2 on /mydocker-1-paravance-10.rennes.grid5000.fr]
 [started TASK: apt install -y iproute2 on /mydocker-0-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-0-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-2-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-4-paravance-10.rennes.grid5000.fr]
changed: [/my

CommandResult(host='/mydocker-0-paravance-10.rennes.grid5000.fr', task='apt install -y iproute2', status='OK', payload={'cmd': 'apt install -y iproute2', 'stdout': "Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  libatm1 libbsd0 libcap2 libcap2-bin libelf1 libmnl0 libpam-cap libxtables12\nSuggested packages:\n  iproute2-doc\nThe following NEW packages will be installed:\n  iproute2 libatm1 libbsd0 libcap2 libcap2-bin libelf1 libmnl0 libpam-cap\n  libxtables12\n0 upgraded, 9 newly installed, 0 to remove and 5 not upgraded.\nNeed to get 1060 kB of archives.\nAfter this operation, 3679 kB of additional disk space will be used.\nGet:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libbsd0 amd64 0.10.0-1 [45.4 kB]\nGet:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libcap2 amd64 1:2.32-1 [15.9 kB]\nGet:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libelf1 amd64 0.176-1.1build1 [44.0 kB]\nGet:4 http://archive.ubuntu.com/ubuntu focal/main amd64 libmnl0 amd64 1.0.4-2 [12.3 kB]\nGet:5 http://archive.ubuntu.com/ubuntu focal/main amd64 libxtables12 amd64 1.8.4-3ubuntu2 [28.4 kB]\nGet:6 http://archive.ubuntu.com/ubuntu focal/main amd64 libcap2-bin amd64 1:2.32-1 [26.2 kB]\nGet:7 http://archive.ubuntu.com/ubuntu focal/main amd64 iproute2 amd64 5.5.0-1ubuntu1 [858 kB]\nGet:8 http://archive.ubuntu.com/ubuntu focal/main amd64 libatm1 amd64 1:2.5.1-4 [21.8 kB]\nGet:9 http://archive.ubuntu.com/ubuntu focal/main amd64 libpam-cap amd64 1:2.32-1 [8352 B]\nFetched 1060 kB in 0s (2883 kB/s)\nSelecting previously unselected package libbsd0:amd64.\r\n(Reading database ... \r(Reading database ... 5%\r(Reading database ... 10%\r(Reading database ... 15%\r(Reading database ... 20%\r(Reading database ... 25%\r(Reading database ... 30%\r(Reading database ... 35%\r(Reading database ... 40%\r(Reading database ... 45%\r(Reading database ... 50%\r(Reading database ... 55%\r(Reading database ... 60%\r(Reading database ... 65%\r(Reading database ... 70%\r(Reading database ... 75%\r(Reading database ... 80%\r(Reading database ... 85%\r(Reading database ... 90%\r(Reading database ... 95%\r(Reading database ... 100%\r(Reading database ... 4961 files and directories currently installed.)\r\nPreparing to unpack .../0-libbsd0_0.10.0-1_amd64.deb ...\r\nUnpacking libbsd0:amd64 (0.10.0-1) ...\r\nSelecting previously unselected package libcap2:amd64.\r\nPreparing to unpack .../1-libcap2_1%3a2.32-1_amd64.deb ...\r\nUnpacking libcap2:amd64 (1:2.32-1) ...\r\nSelecting previously unselected package libelf1:amd64.\r\nPreparing to unpack .../2-libelf1_0.176-1.1build1_amd64.deb ...\r\nUnpacking libelf1:amd64 (0.176-1.1build1) ...\r\nSelecting previously unselected package libmnl0:amd64.\r\nPreparing to unpack .../3-libmnl0_1.0.4-2_amd64.deb ...\r\nUnpacking libmnl0:amd64 (1.0.4-2) ...\r\nSelecting previously unselected package libxtables12:amd64.\r\nPreparing to unpack .../4-libxtables12_1.8.4-3ubuntu2_amd64.deb ...\r\nUnpacking libxtables12:amd64 (1.8.4-3ubuntu2) ...\r\nSelecting previously unselected package libcap2-bin.\r\nPreparing to unpack .../5-libcap2-bin_1%3a2.32-1_amd64.deb ...\r\nUnpacking libcap2-bin (1:2.32-1) ...\r\nSelecting previously unselected package iproute2.\r\nPreparing to unpack .../6-iproute2_5.5.0-1ubuntu1_amd64.deb ...\r\nUnpacking iproute2 (5.5.0-1ubuntu1) ...\r\nSelecting previously unselected package libatm1:amd64.\r\nPreparing to unpack .../7-libatm1_1%3a2.5.1-4_amd64.deb ...\r\nUnpacking libatm1:amd64 (1:2.5.1-4) ...\r\nSelecting previously unselected package libpam-cap:amd64.\r\nPreparing to unpack .../8-libpam-cap_1%3a2.32-1_amd64.deb ...\r\nUnpacking libpam-cap:amd64 (1:2.32-1) ...\r\nSetting up libatm1:amd64 (1:2.5.1-4) ...\r\nSetting up libcap2:amd64 (1:2.32-1) ...\r\nSetting up libcap2-bin (1:2.32-1) ...\r\nSetting up libmnl0:amd64 (1.0.4-2) ...\r\nSetting up libxtables12:amd64 (1.8.4-3ubuntu2) ...\r\nSetting up libbsd0:amd64 (0.10.0-1) 

In [30]:
results = en.run_command("ip a", roles=dockers)
for r in results:
    print(r.host)
    print("-"*20)
    print(r.stdout)


PLAY [all] **********************************************************************************************************************************************************************************

TASK [ip a] *********************************************************************************************************************************************************************************
 [started TASK: ip a on /mydocker-4-paravance-10.rennes.grid5000.fr]
 [started TASK: ip a on /mydocker-3-paravance-10.rennes.grid5000.fr]
 [started TASK: ip a on /mydocker-2-paravance-10.rennes.grid5000.fr]
 [started TASK: ip a on /mydocker-1-paravance-10.rennes.grid5000.fr]
 [started TASK: ip a on /mydocker-0-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-3-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-2-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-4-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-1-paravance-10.rennes.grid5000.fr]
changed: [/mydocker-0-paravance-10.rennes.grid5000.fr

In [31]:
provider.destroy()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from grenoble
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading 1820217 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading enoslib_docker from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Killing the job (rennes, 1820217)


## LXC containers with Distem

https://distem.gitlabpages.inria.fr/

EnOSlib offers a way to use Distem on Grid'5000 using a dedicated provider.
This provider will do the heavy-lifting of 

- reserving the physical resources needed (some machines and a subnet)
- deploying a linux environment (Distem requires it)
- starting the distem server and agents
- makes the initial API calls to the distem server to start the wanted containers.



> Since the Distem server is accessible only from inside Grid'5000, to use this provider it's recommended to launch the script **from inside   
  Grid'5000**. 
  But if you're adventurous and outside Grid'5000, you can try to first create a proxy SOCKS to the frontend of your choice and then set the relevant   variable in your environment (and set `verify: False` in your `~/.python-grid5000.yaml`, and ... restart the kernel. Yes, notebooks are wild sometimes !)

In the following we'll consider being **outside** Grid'5000

In [1]:
# in a terminal: 
# ssh -ND 2100 rennes.grid5000.fr
import os
os.environ["http_proxy"] = "socks5h://localhost:2100"
os.environ["https_proxy"] = "socks5h://localhost:2100"

In [2]:
import logging
from pathlib import Path

import enoslib as en

FORCE = False
CLUSTER = "paravance"

logging.basicConfig(level=logging.DEBUG)
# claim the resources
conf = (
    en.DistemConf
    .from_settings(
        job_name="enoslib_distem",
        force_deploy=FORCE,
        image="file:///home/msimonin/public/distem-stretch.tgz"
    )
    .add_machine(
        roles=["server"],
        cluster=CLUSTER,
        number=1,
        flavour="large"
    )
    .add_machine(
        roles=["client"],
        cluster=CLUSTER,
        number=1,
        flavour="large"
    )
    .finalize()
)

provider = en.Distem(conf)
conf

DEBUG:enoslib.infra.configuration:{
    "job_name": "enoslib_distem",
    "queue": "default",
    "walltime": "02:00:00",
    "force_deploy": false,
    "image": "file:///home/msimonin/public/distem-stretch.tgz",
    "resources": {
        "machines": [
            {
                "undercloud": [],
                "cluster": "paravance",
                "roles": [
                    "server"
                ],
                "flavour_desc": {
                    "core": 4,
                    "mem": 4096
                },
                "number": 1
            },
            {
                "undercloud": [],
                "cluster": "paravance",
                "roles": [
                    "client"
                ],
                "flavour_desc": {
                    "core": 4,
                    "mem": 4096
                },
                "number": 1
            }
        ],
        "networks": [
            "enos_network"
        ]
    }
}


Conf@0x7f05a07435d0
{
    "job_name": "enoslib_distem",
    "queue": "default",
    "walltime": "02:00:00",
    "force_deploy": false,
    "image": "file:///home/msimonin/public/distem-stretch.tgz",
    "resources": {
        "machines": [
            {
                "undercloud": [],
                "cluster": "paravance",
                "roles": [
                    "server"
                ],
                "flavour_desc": {
                    "core": 4,
                    "mem": 4096
                },
                "number": 1
            },
            {
                "undercloud": [],
                "cluster": "paravance",
                "roles": [
                    "client"
                ],
                "flavour_desc": {
                    "core": 4,
                    "mem": 4096
                },
                "number": 1
            }
        ],
        "networks": [
            "enos_network"
        ]
    }
}

In [3]:
roles, networks = provider.init()

DEBUG:enoslib.infra.enos_g5k.g5k_api_utils:Reloading get_all_clusters_sites from cachedir
DEBUG:enoslib.infra.enos_g5k.g5k_api_utils:Reloading get_all_clusters_sites from cachedir
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.grid5000.fr:443
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/rennes HTTP/1.1" 200 3814
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/rennes/clusters/paravance HTTP/1.1" 200 10339
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/rennes/clusters/paravance/nodes HTTP/1.1" 200 485226
DEBUG:enoslib.infra.enos_g5k.g5k_api_utils:Reloading get_all_clusters_sites from cachedir
DEBUG:enoslib.infra.enos_g5k.g5k_api_utils:Reloading get_all_clusters_sites from cachedir
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/rennes HTTP/1.1" 200 3814
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/rennes/clusters/paravance

Waiting for the end of deployment [D-ec237d28-6215-4fb2-9b41-10a3aee46935]


DEBUG:urllib3.connectionpool:Resetting dropped connection: api.grid5000.fr
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/rennes/deployments/D-ec237d28-6215-4fb2-9b41-10a3aee46935 HTTP/1.1" 200 616


Waiting for the end of deployment [D-ec237d28-6215-4fb2-9b41-10a3aee46935]


DEBUG:urllib3.connectionpool:Resetting dropped connection: api.grid5000.fr
DEBUG:urllib3.connectionpool:https://api.grid5000.fr:443 "GET /stable/sites/rennes/deployments/D-ec237d28-6215-4fb2-9b41-10a3aee46935 HTTP/1.1" 200 616


...

DEBUG:enoslib.api:{'hosts': 'all', 'tasks': [{'name': 'kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \\{1,\\}/ /g"|cut -f 2 -d" "`|| true', 'shell': 'kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \\{1,\\}/ /g"|cut -f 2 -d" "`|| true'}, {'name': 'wait_for', 'wait_for': {'state': 'stopped', 'port': 4567}}, {'name': 'wait_for', 'wait_for': {'state': 'stopped', 'port': 4568}}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:{'hosts': 'all', 'tasks': [{'name': 'kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \\{1,\\}/ /g"|cut -f 2 -d" "`|| true', 'shell': 'kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \\{1,\\}/ /g"|cut -f 2 -d" "`|| true'}, {'name': 'wait_for', 'wait_for': {'state': 'stopped', 'port': 4567}}, {'name': 'wait_for', 'wait_for': {'state': 'stopped', 'port': 4568}}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:Using extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ***************************************************************************************************************************************************************

TASK [kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \{1,\}/ /g"|cut -f 2 -d" "`|| true] ****************************************************************************
 [started TASK: kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \{1,\}/ /g"|cut -f 2 -d" "`|| true on paravance-69.rennes.grid5000.fr]
 [started TASK: kill -9 `ps aux|grep "distemd"|grep -v grep|sed "s/ \{1,\}/ /g"|cut -f 2 -d" "`|| true on paravance-72.rennes.grid5000.fr]
changed: [paravance-69.rennes.grid5000.fr]
changed: [paravance-72.rennes.grid5000.fr]

TASK [wait_for] **********************************************************************************************************************************************************
 [started TASK: wait_for on paravance-69.rennes.grid5000.fr]
 [started TASK: wait_for on paravance-72.rennes.grid5000.fr

DEBUG:enoslib.api:{'hosts': 'paravance-69.rennes.grid5000.fr', 'tasks': [{'name': 'file', 'file': {'state': 'directory', 'dest': '/var/log/distem'}}, {'name': 'tmux new-session -d "exec distemd --verbose -d"', 'shell': 'tmux new-session -d "exec distemd --verbose -d"'}, {'name': 'wait_for', 'wait_for': {'state': 'started', 'port': 4567, 'timeout': 10}}, {'name': 'wait_for', 'wait_for': {'state': 'started', 'port': 4568, 'timeout': 10}}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:{'hosts': 'paravance-69.rennes.grid5000.fr', 'tasks': [{'name': 'file', 'file': {'state': 'directory', 'dest': '/var/log/distem'}}, {'name': 'tmux new-session -d "exec distemd --verbose -d"', 'shell': 'tmux new-session -d "exec distemd --verbose -d"'}, {'name': 'wait_for', 'wait_for': {'state': 'started', 'port': 4567, 'timeout': 10}}, {'name': 'wait_for', 'wait_for': {'state': 'started', 'port': 4568, 'timeout': 10}}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:Using ex


PLAY [paravance-69.rennes.grid5000.fr] ***********************************************************************************************************************************

TASK [file] **************************************************************************************************************************************************************
 [started TASK: file on paravance-69.rennes.grid5000.fr]
changed: [paravance-69.rennes.grid5000.fr]

TASK [tmux new-session -d "exec distemd --verbose -d"] *******************************************************************************************************************
 [started TASK: tmux new-session -d "exec distemd --verbose -d" on paravance-69.rennes.grid5000.fr]
changed: [paravance-69.rennes.grid5000.fr]

TASK [wait_for] **********************************************************************************************************************************************************
 [started TASK: wait_for on paravance-69.rennes.grid5000.fr]
ok: [par

DEBUG:root:method=post, route=http://paravance-69.rennes.grid5000.fr:4567/pnodes/, data={'target': ['paravance-69.rennes.grid5000.fr', 'paravance-72.rennes.grid5000.fr'], 'desc': {}, 'async': False}
DEBUG:urllib3.connectionpool:Starting new HTTP connection (2): paravance-69.rennes.grid5000.fr:4567
DEBUG:urllib3.connectionpool:http://paravance-69.rennes.grid5000.fr:4567 "POST /pnodes/ HTTP/1.1" 200 14894
DEBUG:root:method=post, route=http://paravance-69.rennes.grid5000.fr:4567/vnetworks, data={'name': 'enoslib_distem_network', 'address': '10.158.0.0/22'}
DEBUG:urllib3.connectionpool:http://paravance-69.rennes.grid5000.fr:4567 "POST /vnetworks HTTP/1.1" 200 157
DEBUG:root:method=post, route=http://paravance-69.rennes.grid5000.fr:4567/vnodes/vnode-0, data={'desc': {'host': 'paravance-69.rennes.grid5000.fr'}, 'ssh_key': {'public': 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDd5AKmSuvzd8zNqTKOcZ6RTiQ+xvMTaVjU+m1ptRV75YU2hgyRffT/vOi047iuUyxThtDhBhCHb7da+9QmNU+mQvXM9SoTzw6M+6sX4R2fold7K5U/v/HDp6zl

In [4]:
roles

{'server': [Host(address='10.158.0.1', alias='10.158.0.1', user='root', keyfile='/home/msimonin/workspace/repos/enoslib/docs/jupyter/keys/id_rsa', port=None, extra={'gateway': 'paravance-69.rennes.grid5000.fr', 'gateway_user': 'root'}, net_devices=set())], 'client': [Host(address='10.158.0.2', alias='10.158.0.2', user='root', keyfile='/home/msimonin/workspace/repos/enoslib/docs/jupyter/keys/id_rsa', port=None, extra={'gateway': 'paravance-69.rennes.grid5000.fr', 'gateway_user': 'root'}, net_devices=set())]}

In [5]:
networks

{'__subnet__': [<enoslib.infra.enos_g5k.objects.G5kEnosSubnetNetwork object at 0x7f05799cb190>]}

In [6]:
roles = en.sync_info(roles, networks)

DEBUG:enoslib.api:{'hosts': 'all', 'tasks': [{'name': 'hostname', 'raw': 'hostname'}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:{'hosts': 'all', 'tasks': [{'name': 'hostname', 'raw': 'hostname'}], 'gather_facts': False, 'strategy': 'linear'}
DEBUG:enoslib.api:Using extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ***************************************************************************************************************************************************************

TASK [hostname] **********************************************************************************************************************************************************
 [started TASK: hostname on 10.158.0.1]
 [started TASK: hostname on 10.158.0.2]
changed: [10.158.0.2]
changed: [10.158.0.1]


DEBUG:enoslib.api:Using extra_vars = {'enos_action': 'check_network', 'facts_file': '/home/msimonin/workspace/repos/enoslib/docs/jupyter/_tmp_enos_/facts.json', 'ansible_python_interpreter': 'python3'}
DEBUG:enoslib.api:Running playbook /home/msimonin/workspace/repos/enoslib/enoslib/ansible/utils.yml with vars:
{'enos_action': 'check_network', 'facts_file': '/home/msimonin/workspace/repos/enoslib/docs/jupyter/_tmp_enos_/facts.json', 'ansible_python_interpreter': 'python3'}



PLAY [Gather facts for all hosts] ****************************************************************************************************************************************

TASK [Gathering Facts] ***************************************************************************************************************************************************
ok: [10.158.0.1]
ok: [10.158.0.2]

TASK [setup] *************************************************************************************************************************************************************
ok: [10.158.0.2]
ok: [10.158.0.1]

PLAY [Utils functions] ***************************************************************************************************************************************************

TASK [Gathering Facts] ***************************************************************************************************************************************************
ok: [10.158.0.1]
ok: [10.158.0.2]

TASK [utils : include] **************

In [8]:
results = en.run_command("nproc", roles=roles)
[(r.host,  r.stdout) for r in results]

DEBUG:enoslib.api:{'hosts': 'all', 'gather_facts': False, 'tasks': [{'name': 'nproc', 'shell': 'nproc', 'args': {}}]}
DEBUG:enoslib.api:Using extra_vars = {'ansible_python_interpreter': 'python3'}



PLAY [all] ***************************************************************************************************************************************************************

TASK [nproc] *************************************************************************************************************************************************************
 [started TASK: nproc on 10.158.0.1]
 [started TASK: nproc on 10.158.0.2]
changed: [10.158.0.1]
changed: [10.158.0.2]


[('10.158.0.1', '32'), ('10.158.0.2', '32')]

## Conclusion
 TODO